<a href="https://colab.research.google.com/github/cosmina98/PhD/blob/main/Work_in_progress/MessPass_GIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install plotnine --quiet
!pip install torch_geometric
! install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 2.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=ec89ecd6aaf79bfae711079f23cdcae9df0c5ee2abedbb58951f03cf591c6337
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
install: missing destination file operand after 'torch_geometric'
Try 'install --help' for more information.
     |████████████████████████████████| 7.9 MB 2.3 MB/s 
     |████████████████████████████████| 3.5 MB 2.1 MB/s 


In [6]:
import torch_geometric
from torch_geometric.datasets import Planetoid
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [5]:
import torch_geometric 
from typing import Callable, List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch import Tensor
from torch.optim import Optimizer
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv,ChebConv
#from torch_geometric.utils import accuracy
from typing_extensions import Literal, TypedDict
from torch_geometric.utils import get_laplacian
from torch_geometric.utils import add_self_loops, degree
import numpy as np
import pandas as pd
from plotnine import ggplot, geom_line, aes, xlab, theme, element_blank, ggtitle
import scipy.sparse as sparse
from sklearn.model_selection import KFold
from torch_geometric.utils import to_scipy_sparse_matrix
from dataclasses import dataclass

In [4]:
# libary imports 
import os
import torch
from torch import nn
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn.metrics import accuracy_score

from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import time
import pandas as pd

import itertools
from functools import reduce
import operator
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
from torch.utils.data import Subset

from torch.optim.adadelta import Adadelta
from torch.optim.adagrad import Adagrad
from torch.optim.adam import Adam
from torch.optim.adamax import Adamax
from torch.optim.adamw import AdamW
from torch.optim.optimizer import Optimizer
from torch.optim.sgd import SGD

import torch_geometric 
from typing import Callable, List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch import Tensor
from torch.optim import Optimizer
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv,ChebConv
from typing_extensions import Literal, TypedDict
from torch_geometric.utils import get_laplacian
import numpy as np
import pandas as pd
from plotnine import ggplot, geom_line, aes, xlab, theme, element_blank, ggtitle
import scipy.sparse as sparse
from sklearn.model_selection import KFold
from torch_geometric.utils import to_scipy_sparse_matrix
from dataclasses import dataclass
import torch_geometric.utils
#from torch_geometric.utils import accuracy


In [7]:
device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')


#Dataset imports

# Training and testing targets for each dataset

In [8]:
dataset=list(range(3))
i=0
for dataset_name in ['Cora','CiteSeer','PubMed']:
    dataset[i] = Planetoid('/tmp/dataset_name', name=dataset_name)
    num_nodes = dataset[i].data.num_nodes
    num_edges = dataset[i].data.num_edges // 2
    train_len = dataset[i][0].train_mask.sum()
    val_len = dataset[i][0].val_mask.sum()
    test_len = dataset[i][0].test_mask.sum()
    other_len = num_nodes - train_len - val_len - test_len
    print(f"Dataset: {dataset[i].name}")
    print(f"Num. nodes: {num_nodes} (train={train_len}, val={val_len}, test={test_len}, other={other_len})")
    print(f"Num. edges: {num_edges}")
    print(f"Num. node features: {dataset[i].num_node_features}")
    print(f"Num. classes: {dataset[i].num_classes}")
    print(f"Dataset len.: {dataset[i].len()}")
    i=i+1
    print(" ")


Processing...
Done!


Dataset: Cora
Num. nodes: 2708 (train=140, val=500, test=1000, other=1068)
Num. edges: 5278
Num. node features: 1433
Num. classes: 7
Dataset len.: 1
 


Processing...
Done!


Dataset: CiteSeer
Num. nodes: 3327 (train=120, val=500, test=1000, other=1707)
Num. edges: 4552
Num. node features: 3703
Num. classes: 6
Dataset len.: 1
 


Dataset: PubMed
Num. nodes: 19717 (train=60, val=500, test=1000, other=18157)
Num. edges: 44324
Num. node features: 500
Num. classes: 3
Dataset len.: 1
 


Processing...
Done!


In [9]:
d={0:{}, 1:{}, 2:{}}
for i in range(3):
    data=dataset[i]
    X_train=data[0].x[data[0].train_mask]
    d[i]['X_train']=X_train
    y_train=data[0].y[data[0].train_mask]
    d[i]['y_train']=y_train
    X_test=data[0].x[data[0].test_mask]
    d[i]['X_test']=X_test
    y_test=data[0].y[data[0].test_mask]
    d[i]['y_test']=y_test
    X_val=data[0].x[data[0].val_mask]
    d[i]['X_val']=X_val
    y_val=data[0].y[data[0].val_mask]
    d[i]['y_val']=X_val


#Message Passing 

In [10]:
from torch_scatter import scatter_mean

dataset[0].data
x=dataset[0].data.x
edge_index=dataset[0].data.edge_index
x_j = x[edge_index[0]]  # Source node features [num_edges, num_features]
x_i = x[edge_index[1]]  # Target node features [num_edges, num_features]

In [11]:
torch.manual_seed(0)


class GIN(nn.Module):
    
    def __init__(self, in_channels, out_channels):
        super().__init__()  
        #initialisation
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.flow = 'source_to_target'
        self.out_channels=out_channels
        self.reset_parameters()


        self.W1 = nn.Parameter(torch.zeros(size=(in_channels, out_channels))) #xavier paramiter inizializator
        nn.init.xavier_uniform_(self.W1.data, gain=1.414)
      

    def mean_aggregate(self,x,edge_index):
        row,col=edge_index
        tmp = torch.index_select(x, 0, col) # shape [num_edges, embed_size)
        aggr2 = scatter_mean(tmp,row,0)
        return aggr2

    def reset_parameters(self):
        self.lin.reset_parameters()
       

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        num_nodes=len(x)
        # Step 1: Linearly transform node feature matrix.
        #first term
        first_term = self.lin(x)
        
        # Step 4-5: Start propagating messages.
        out=self.collect(x,edge_index)
        x_j=out['x_j']
        out = self.message(x_j)
        second_term= self.mean_aggregate(out,edge_index)
        out=first_term + self.lin(second_term)
        
        return out

    def collect(self ,x,edge_index):
        i, j = (1, 0) if self.flow == 'source_to_target' else (0, 1)
        out={}
        # 2. construct message x_j, x_i. Both with shape [num_edge, embed_size]
        out['x_j'] = x.index_select(0, edge_index[i]) 
        out['x_i'] = x.index_select(0, edge_index[j])
        out['edge_index_i'] = edge_index[i] # Source node edges 
        out['edge_index_j'] = edge_index[j]   # Target node edges 
        return out  

   
    def message(self, x_j):
        # x_j has shape [E, out_channels]
        return  x_j
    
    #'not needed'
    def update(self, aggr_out):
    # aggr_out has shape [num_nodes, out_channels]

    # Step 5: Return new node embeddings.
        return aggr_out

In [12]:
conv = GIN(x.shape[1], 256)
m=conv(x, edge_index)
m

tensor([[-0.0012,  0.0550, -0.1156,  ..., -0.0228,  0.0786,  0.0417],
        [-0.0771,  0.1365, -0.0750,  ..., -0.0700,  0.0899, -0.0674],
        [ 0.0017,  0.1060, -0.1788,  ..., -0.0507,  0.0400,  0.0582],
        ...,
        [ 0.0596,  0.0694, -0.0410,  ..., -0.0573,  0.1741,  0.0983],
        [-0.0717,  0.1124, -0.1071,  ..., -0.0689,  0.0135, -0.1141],
        [-0.0809,  0.0832, -0.0549,  ...,  0.0109,  0.0957,  0.0488]],
       grad_fn=<AddBackward0>)

In [ ]:
class GINNet(torch.nn.Module):
    def __init__(self,num_node_features, num_classes):
       
        super(GINNet, self).__init__()

        num_features = num_node_features
        num_classes=num_classes
        dim = 32

        nn1 = Sequential(Linear(num_features, dim), ReLU(), Linear(dim, dim))
        self.conv1 = GINConv(nn1)
        self.bn1 = torch.nn.BatchNorm1d(dim)

        nn2 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv2 = GINConv(nn2)
        self.bn2 = torch.nn.BatchNorm1d(dim)

        nn3 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv3 = GINConv(nn3)
        self.bn3 = torch.nn.BatchNorm1d(dim)

        nn4 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv4 = GINConv(nn4)
        self.bn4 = torch.nn.BatchNorm1d(dim)

        nn5 = Sequential(Linear(dim, dim), ReLU(), Linear(dim, dim))
        self.conv5 = GINConv(nn5)
        self.bn5 = torch.nn.BatchNorm1d(dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim,num_classes)

    def forward(self, x, edge_index, batch=None):
        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        x = F.relu(self.conv5(x, edge_index))
        x = self.bn5(x)
        #x = global_add_pool(x, batch=0)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [ ]:
model=GINNet(dataset[0].num_node_features, dataset[0].num_classes).to(device)
print(GINNet(dataset[0].num_node_features, dataset[0].num_classes))
print(GINNet(dataset[1].num_node_features, dataset[1].num_classes))
print(GINNet(dataset[2].num_node_features, dataset[2].num_classes))